In [12]:
import skmob
import pandas as pd
from skmob.preprocessing import filtering,  detection, clustering
from skmob.measures.individual import location_frequency
from skmob.measures.collective import mean_square_displacement, visits_per_location
import os
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pyproj import Transformer
from shapely.ops import transform
import numpy as np
from math import sin, cos, sqrt, atan2, radians, floor, ceil, sqrt
import geopy as geo

In [21]:
#GLOBAL VARIABLE AND FUNCTION
dim_cell_m=1
freq='1h'
users=[]
#ADD ROW FOR DEBUG USER INSIDE CELLS
def add_row(df,element):
    df = pd.concat([df, pd.DataFrame.from_records([element])])
    return df
    

#READ DATASET
def read_dataset():
    #url = skmob.utils.constants.GEOLIFE_SAMPLE
    df = pd.read_csv("../dataset/geolife_example.csv", sep=',')
    users = df["uid"].unique()
    return df

In [17]:


df = read_dataset()
### GEOLIFE DATASET ANALYSIS
for i in range(0,3):
    df = add_row(df,{'lat':  40.071316361318885 , 'lon': 117.0547972535935, 'uid': 100, 'tid': 18000, 'date_time': '2009-05-08 07:46:19.770112'})

tdf = skmob.TrajDataFrame(df, latitude='lat', longitude='lon', user_id='user', datetime='date_time')

df["index"] = df.index
print(df.head(1))


         lat         lon   uid    tid                   date_time  index
0  40.030941  116.341601  3000  18670  2009-10-30 03:47:00.801167      0


In [18]:
from datetime import datetime
import matplotlib.pyplot as plt
df = read_dataset()


df["index"] = df.index
print(df.shape)
df["date"] = df["date_time"].apply(lambda row: row.split(" ")[0])

df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d" )

mask = ((df['date'] > pd.to_datetime("2008-07-01",format = "%Y-%m-%d" )) & (df['date'] <= pd.to_datetime("2009-09-30",format = "%Y-%m-%d" )  ) ) 
print(df.shape)
df = df.loc[mask]
df = df.drop(columns=['date'])
print(df.shape)


(769054, 6)
(769054, 7)
(631559, 6)


In [19]:
### TAKE BORDER COORDINATION OF THE REGION

unique_coordinates = tdf[["lat","lng"]].drop_duplicates()

min_lat, max_lat = min(unique_coordinates["lat"]),max(unique_coordinates["lat"])
min_lon, max_lon = min(unique_coordinates["lng"]),max(unique_coordinates["lng"])

top_left = (max_lat,max_lon)
top_right = (max_lat,min_lon)
bottom_left = (min_lat,max_lon)
bottom_right = (min_lat,min_lon)

print(top_left,top_right)
print(bottom_left,bottom_right)

(40.307909499999994, 117.1137114) (40.307909499999994, 115.74167620000001)
(39.53285629999999, 117.1137114) (39.53285629999999, 115.74167620000001)


In [25]:
import geopy.distance
dim_cell_m=2
R = 6373.0

lat1 = radians(max_lat)
lat2 = radians(min_lat)
lon1 = radians(max_lon)
lon2 = radians(max_lon)

dlon = lon2 - lon1
dlat = lat2 - lat1


a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c


height = distance 
print("Max distance considering latitude", height)
print(geo.distance.distance((min_lat,min_lon),(max_lat,min_lon)).km)


lat1 = radians(min_lat)
lon1 = radians(min_lon)
lat2 = radians(min_lat)
lon2 = radians(max_lon)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c



coords_1 = (lat1, lon1)
coords_2 = (lat2, lon2)



length = distance 
print("Max longitude considering longitude", length)
print(geo.distance.distance((min_lat,min_lon),(min_lat,max_lon)).km)
print("Total area region (m^2): ",height*length)

cells_lat = height/dim_cell_m + 1
cells_lon= length/dim_cell_m + 1
print("Each cell as dimension")
print(height/cells_lat,length/cells_lon)
print("#Cell lat (rows), #Cell lon (columns)")
print(floor(cells_lat),floor(cells_lon))



Max distance considering latitude 86.20903818006724
86.0565641834225
Max longitude considering longitude 117.70176427180961
117.9567132965088
Total area region (m^2):  10146.95588996971
Each cell as dimension
1.95465317293411 1.9665836170057016
#Cell lat (rows), #Cell lon (columns)
44 59


In [26]:
print(top_left, top_right)
print(bottom_left,bottom_right)
print(floor(cells_lat),floor(cells_lon))


cols = np.linspace(bottom_left[1], bottom_right[1], num=floor(cells_lon))
rows = np.linspace(bottom_left[0], top_left[0], num=floor(cells_lat))
diff=0
i=0
R = 6373.0
df_grid = df

df_grid["col"] = df["lon"]
def calc_col(row):
    tmp = []
    if((row["index"]%100000)==0):
        print(row["index"])
    for col in cols:
        


            lat1 = radians(row["lat"])
            lon1 = radians(col)
            lat2 = radians(row["lat"])
            lon2 = radians(row["lon"])
            coords_1 = (lat1, lon1)
            coords_2 = (lat2, lon2)
            distance = geopy.distance.geodesic(coords_1, coords_2).km
          
            tmp.append(distance)
    return np.argmin(tmp)

def calc_row(row_):
    tmp = []
    if((row_["index"]%100000)==0):
        print(row_["index"])
    for row in rows:
            lat1 = radians(row)
            lon1 = radians(row_["lon"])
            lat2 = radians(row_["lat"])
            lon2 = radians(row_["lon"])
            coords_1 = (lat1, lon1)
            coords_2 = (lat2, lon2)
            distance = geopy.distance.geodesic(coords_1, coords_2).km
          
            tmp.append(distance)
    return np.argmin(tmp)



df_grid["col"]= df_grid.apply(calc_col, axis=1)
df_grid['row'] = df_grid.apply(calc_row, axis=1)

(40.307909499999994, 117.1137114) (40.307909499999994, 115.74167620000001)
(39.53285629999999, 117.1137114) (39.53285629999999, 115.74167620000001)
44 59
100000
200000
300000
400000
700000
100000
200000
300000
400000
700000


In [29]:
print(len(df_grid["row"].unique()),len(df_grid["col"].unique()))

44 59


In [30]:
import numpy as np
import random
value = {0:0,1:0,2:0}
p=[0.34, 0.33,0.33] # big_work, #medium_work, #small_work
workload = []
while True:
    workload = []
    for i in range(len(df)):
    #creates one number out of 0 or 1 with prob p 0.5 for 0 and 0.5 for 1
        test = np.random.choice(np.arange(0, 3), p=p)
        workload.append(test)
        value[test]+=1
    print("generated..")
    check=False
    for i in range(len(value)):
        if( ((round(value[i]/len(df),2)*100)/100 )!=p[i]):
            print(value[i]/len(df))
            check=True
            break
    if(check==False):
        break
    else:
        value = {0:0,1:0,2:0}
print(value)
df_work = df_grid
df_work["work"] = workload
df_work.to_csv("../dataset/proc_geolife_tot_"+str(dim_cell_m)+"_m.csv")

generated..
{0: 215120, 1: 207283, 2: 209156}


In [32]:
import datetime
df_work = pd.read_csv("../dataset/proc_geolife_tot_"+str(dim_cell_m)+"_m.csv")
qgis = pd.DataFrame()
# ADDING ROW FOR DEBUG 
# for i in range(0,3):
#     df_work = add_row(df_work,{'lat':  40.19918568222689 , 'lon': 40.19723074921124, 'uid': 100, 'tid': 18000, 'date_time': '2008-07-01 09:00:00', 'col':0.0, 'row':148, 'work':2})


 

def count_work_per_cell_and_time(df_cell):
    df_final = pd.DataFrame()
    for group_id, group_df in df_cell.groupby(["col","row"]):
        group_df["date_hour"] = pd.to_datetime(group_df['date_time'], format = "%Y-%m-%d %H:%M:%S" ).astype('datetime64[s]') #%Y-%m-%d %H:%M:%S / datetime64[h/m/s]
        
        #row = {'lats':0,'lons':0,'uids':0,'small':0,'big':0,'row':0,'col':0,'date':"", 'work':0}
        for group_id2, group_date in group_df.groupby(pd.Grouper(key='date_hour', freq='1h')):
                if(len(group_date)>=1):
                    date = pd.to_datetime(group_date["date_time"], format = "%Y-%m-%d %H" ).astype('datetime64[h]').unique()[0]
                   
                    
                    works = [0,0,0]
                    for _,group_work in group_date.groupby("work"):
                          works[group_work["work"].unique()[0]] = group_work["work"].count()
                    medoids = group_date[["lat","lon"]].mean(axis=0)
                    row=group_date["row"].unique()[0]
                    col=group_date["col"].unique()[0]
                    df_final = add_row(df_final,
                    {
                         'users': len(group_date["uid"].unique()),
                         'work_1': works[0],'work_2': works[1],'work_3': works[2],
                         "center_lat": medoids["lat"], "center_lon": medoids["lon"],'row':row,'col':col,
                         'date': date,
                    
                                        
                    })
                    
           
    return df_final

# df_without_date = count_work_per_cell(df_work)
# print(df_without_date)
df_date = count_work_per_cell_and_time(df_work)
print(df_date.shape)
print(df_date.head(1))
df_date.to_csv("../dataset/qgis_"+str(dim_cell_m)+"_"+freq+".csv")

(37661, 9)
   users  work_1  work_2  work_3  center_lat  center_lon  row  col  \
0      1       2       3       4   40.079001  117.102987   30    0   

                 date  
0 2008-07-20 19:00:00  
